In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Some tensorflow warnings in the pipeline we inspect
from mlinspect.utils import get_project_root
from example_pipelines.healthcare import custom_monkeypatching

from mlinspect import PipelineInspector
from mlinspect.inspections import MaterializeFirstOutputRows

HEALTHCARE_FILE_PY = os.path.join(str(get_project_root()), "examples", "explainability", "example_pipeline.py")

inspector_result = PipelineInspector\
    .on_pipeline_from_py_file(HEALTHCARE_FILE_PY) \
    .add_custom_monkey_patching_module(custom_monkeypatching) \
    .add_required_inspection(MaterializeFirstOutputRows(5)) \
    .execute()

extracted_dag = inspector_result.dag
dag_node_to_inspection_results = inspector_result.dag_node_to_inspection_results
check_results = inspector_result.check_to_check_results

In [1]:
from IPython.display import Image
from mlinspect.visualisation import save_fig_to_path

filename = os.path.join(str(get_project_root()), "examples", "explainability", "explainability.png")
save_fig_to_path(extracted_dag, filename)

Image(filename=filename)

In [ ]:
from IPython.display import display

check_result_df = PipelineInspector.check_results_as_data_frame(check_results)
display(check_result_df)

In [ ]:
from IPython.display import display

relevant_nodes = [node for node in extracted_dag.nodes if node.node_id in {17, 18}]

for dag_node in relevant_nodes:
    print("\n\033[1m{} ({})\033[0m\n{}\n{}".format(
        dag_node.operator_info.operator,
        dag_node.details.description,
        dag_node.optional_code_info.source_code,
        dag_node.optional_code_info.code_reference))
    materialize_inspection_result = dag_node_to_inspection_results[dag_node][MaterializeFirstOutputRows(5)]
    display(materialize_inspection_result)